<a href="https://colab.research.google.com/github/MunJinSeo/MachineLearning/blob/main/%EA%B8%B0%EA%B3%84%ED%95%99%EC%8A%B5(AAI107)_Team13_%EA%B8%88%EC%9C%B5%EB%89%B4%EC%8A%A4_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_%EB%AA%A8%EB%8D%B8%ED%99%95%EC%9D%B8_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 금융 뉴스 감정 분석 : 모델 동작 확인 Test
- 사용 모델: snunlp/KR-FinBert-SC
- https://huggingface.co/snunlp/KR-FinBert-SC
- https://github.com/snunlp/KR-FinBert-SC

### 기본 환경 설치

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
     |████████████████████████████████| 596 kB 48.8 MB/s 
     |████████████████████████████████| 6.6 MB 47.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification # pre-trained된 모델 로딩
import torch

import urllib.request # 데이터 파일 URL다운로드
from tqdm import tqdm #모델 예측시 계산 진행상태를 보여줌

#from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset # 데이터셋을 분할하여 사용

import gc #GPU 캐쉬 초기화

In [ ]:
print(torch.cuda.is_available() ) # GPU가용성 확인
print(torch.cuda.current_device() ) #현재 선택한 장치의 인덱스 
print(torch.cuda.device_count() ) #사용할 수 있는 GPU 수
print(torch.cuda.get_device_name(0) ) #장치 이름

True
0
1
Tesla T4


In [ ]:
# GPU or CPU
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
  device = torch.device("cpu")
  print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# GPU 캐쉬 비워줌
gc.collect()
torch.cuda.empty_cache()

### 데이터 준비

In [ ]:
class Dataset_test(Dataset):
  def __init__(self, file_name):
    # --- 전처리 1 start ------------------------
    # 자료 출처 : 빅카인즈 https://www.bigkinds.or.kr/
    #  빅카인즈 뉴스 자료는 해당 사이트에서 수작업으로 추출했으며, 해당 파일을 나의 github에 올려 놓은 후 아래 감성분석에 사용함)
    #urllib.request.urlretrieve("https://raw.githubusercontent.com/MunJinSeo/MachineLearning/main/samsung_news_20180101-20181231.xlsx", filename="samsung_news_2018.xlsx")
    
    #self.dataset = pd.read_csv(file_name, sep=',')
    test_data = ["현대바이오, '폴리탁셀' 코로나19 치료 가능성에 19% 급등" #긍정
                 ,"이수화학, 3분기 영업익 176억…전년比 80%↑" #긍정
                 ,"GKL, 7년 만에 두 자릿수 매출성장 예상" #긍정
                 ,"위지윅스튜디오, 콘텐츠 활약에 사상 첫 매출 1000억원 돌파" #긍정
                 ,"삼성전자, 2년 만에 인도 스마트폰 시장 점유율 1위 '왕좌 탈환'" #긍정
                 ,"영화관株 '코로나 빙하기' 언제 끝나나…CJ CGV 올 4000억 손실 날수도" #부정
                 ,"C쇼크에 멈춘 흑자비행…대한항공 1분기 영업적자 566억" #부정
                 ,"'1000억대 횡령·배임' 최신원 회장 구속… SK네트웍스 경영 공백 방지 최선" #부정
                 ,"부품 공급 차질에…기아차 광주공장 전면 가동 중단" #부정
                 ,"현대제철, 지난해 영업익 3,313억원···전년比 67.7% 감소" #부정
                 ]
    self.dataset = pd.DataFrame({"제목":test_data})
    

    # tokenizer
    self.tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")
    
    # --- 전처리 1 end ------------------------
  
  def __len__(self):
    return len(self.dataset)
  
  def clean_text(self, txt):
    # --- 전처리 2 start ----------------------
    cleaned = self.sub1.sub('', txt.strip())  # .strip()은 문장의 앞뒤 공백제거함
    cleaned = self.sub2.sub(' ', cleaned)
    cleaned = self.sub3.sub('.', cleaned)
    #cleaned = emoticon_normalize(cleaned, num_repeats=3) # 감정 반복 단순화
    #cleaned = repeat_normalize(cleaned, num_repeats=2) # 중복 글자 단순화
    #cleaned = only_text(cleaned) # text만 추출
    #cleaned = only_hangle(cleaned) # 한글만 추출
    #cleaned = only_hangle_number(cleaned) # 한글/숫자만 추출

    #if len(ssStr) > 1:
    cleaned = "[CLS] " + cleaned + " [SEP]"
    # --- 전처리 2 end ------------------------
    return cleaned

  
  def __getitem__(self, idx):

    text = self.dataset['제목'][idx] #제목, 본문 각각 해봐야할듯함

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True, #`pad_to_max_length` 인수는 더 이상 사용되지 않으며 향후 버전에서 제거. 일괄 처리에서 가장 긴 시퀀스로 채우려면 `padding=True` 또는 `padding='longest'`를 사용하거나 `padding='max_length'를 사용하세요. ` 최대 길이로 채웁니다. 이 경우 `max_length`로 특정 길이를 지정하거나(예: `max_length=45`) max_length를 None으로 두어 모델의 최대 입력 크기(예: Bert의 경우 512)로 채울 수 있습니다.
        #padding=True ,
        add_special_tokens=True
        )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask

In [ ]:
sample_dataset = Dataset_test("")
sample_dataset.dataset

### 모델 생성 : snunlp/KR-FinBert-SC

In [ ]:
# 모델 생성 : snunlp/KR-FinBert-SC
# https://huggingface.co/snunlp/KR-FinBert-SC
# https://github.com/snunlp/KR-FinBert-SC

model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC",num_labels=3).to(device)
#model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert",num_labels=3)
#model.cuda()

### 데이터 로딩 및 배치사이즈 정의

In [ ]:
# 데이터 로딩
batchSize = 30 # GPU 메모리 부족 오류나면 수치를 줄일것

sample_loader = DataLoader(sample_dataset, batch_size=batchSize, shuffle=False)

sample_result = sample_dataset.dataset.copy(deep=True)
print(sample_result)

### 모델 실행 및 감성예측

In [ ]:
model.eval() #모델 평가모드로 변경

# GPU 캐쉬 비워줌
gc.collect()
torch.cuda.empty_cache()

idx_s = 0
idx_e = 0

#label_dict = {0: 'negative', 1: 'neutral', 2: 'positive'}
print(model.config.id2label) # 모델의 분류 종류 보기

#예측 결과 초기화
sample_result['pred_label'] = 0 # 예측 분류값
sample_result['negative'] = 0 # 부정적 감성 확률
sample_result['neutral'] = 0 # 보통 확률
sample_result['positive'] = 0 # 긍정적 감성 확률

# 배치 사이즈별 예측 실행
# tqdm은 진행상황을 표시해줌
#for input_ids_batch, attention_masks_batch,y_batch in tqdm(sample_loader):
for input_ids_batch, attention_masks_batch in tqdm(sample_loader):
  #y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1) #예측한 원소중 max값 1개만 반환
  predictions = torch.nn.functional.softmax(y_pred, dim=-1) # 예측한 원소의 값이 0과 1사이의 값으로 변환 (원소 합은 1)
  #print(predictions)
  
  rsList = list(map(int, predicted)) # 결과를 한번에 저장하기 위해 LIST로 변환 처리
  rs_negative = predictions[:, 0].tolist() #부정
  rs_neutral = predictions[:, 1].tolist() #보통
  rs_positive = predictions[:, 2].tolist() #긍정

  print(rsList)
  idx_e += len(rsList) #해당 배치구간내에 index 끝값 계산
  idx_s = idx_e - len(rsList)
  print("index==", idx_s, idx_e)

  sample_result['pred_label'][idx_s : idx_e] = rsList #배치사이즈 만큼 예측된 값 일괄 저장
  sample_result['negative'][idx_s : idx_e] = rs_negative
  sample_result['neutral'][idx_s : idx_e] = rs_neutral
  sample_result['positive'][idx_s : idx_e] = rs_positive


In [ ]:
print(sample_result)